In [4]:
#查看图片形状
image_path = r"D:\demo\gt\3.png"
import cv2
import numpy as np

# 读取图片
image = cv2.imread(image_path)

# 检查是否成功读取图片
if image is None:
    print("无法读取图片，请检查路径")
else:
    # 获取图片的通道信息
    channels = cv2.split(image)

    # 显示原始图片
    cv2.imshow('Original Image', image)

    # 显示每个通道的信息
    for i, channel in enumerate(channels):
        cv2.imshow(f'Channel {i+1}', channel)

    # 等待用户按下任意键后关闭窗口
    cv2.waitKey(0)
    cv2.destroyAllWindows()



In [8]:
import cv2
import numpy as np
from os import path as osp

from basicsr.metrics import calculate_psnr, calculate_ssim
from basicsr.utils import bgr2ycbcr, scandir


def calculate_psnr_ssim(gt_path, restored_path, crop_border=0, suffix='', test_y_channel=False, correct_mean_var=False):
    """Calculate PSNR and SSIM for images.

    Args:
        gt_path (str): Path to gt (Ground-Truth).
        restored_path (str): Path to restored images.
        crop_border (int): Crop border for each side.
        suffix (str): Suffix for restored images.
        test_y_channel (bool): If True, test Y channel (In MatLab YCbCr format). If False, test RGB channels.
        correct_mean_var (bool): Correct the mean and var of restored images.
    """
    psnr_all = []
    ssim_all = []
    img_list_gt = sorted(list(scandir(gt_path, recursive=True, full_path=True)))
    img_list_restored = sorted(list(scandir(restored_path, recursive=True, full_path=True)))

    if test_y_channel:
        print('Testing Y channel.')
    else:
        print('Testing RGB channels.')

    for i, img_path in enumerate(img_list_gt):
        basename, ext = osp.splitext(osp.basename(img_path))
        img_gt = cv2.imread(img_path, cv2.IMREAD_UNCHANGED).astype(np.float32) / 255.
        # 单通道图片，转为三通道
        if img_gt.ndim == 2:
            img_gt = np.repeat(np.expand_dims(img_gt, axis=2), 3, axis=2)
        if suffix == '':
            img_path_restored = img_list_restored[i]
        else:
            img_path_restored = osp.join(restored_path, basename + suffix + ext)
        img_restored = cv2.imread(img_path_restored, cv2.IMREAD_UNCHANGED).astype(np.float32) / 255.
        # 单通道图片，转为三通道
        if img_restored.ndim == 2:
            img_restored = np.repeat(np.expand_dims(img_restored, axis=2), 3, axis=2)

        if correct_mean_var:
            mean_l = []
            std_l = []
            for j in range(3):
                mean_l.append(np.mean(img_gt[:, :, j]))
                std_l.append(np.std(img_gt[:, :, j]))
            for j in range(3):
                # correct twice
                mean = np.mean(img_restored[:, :, j])
                img_restored[:, :, j] = img_restored[:, :, j] - mean + mean_l[j]
                std = np.std(img_restored[:, :, j])
                img_restored[:, :, j] = img_restored[:, :, j] / std * std_l[j]

                mean = np.mean(img_restored[:, :, j])
                img_restored[:, :, j] = img_restored[:, :, j] - mean + mean_l[j]
                std = np.std(img_restored[:, :, j])
                img_restored[:, :, j] = img_restored[:, :, j] / std * std_l[j]

        if test_y_channel and img_gt.ndim == 3 and img_gt.shape[2] == 3:
            img_gt = bgr2ycbcr(img_gt, y_only=True)
            img_restored = bgr2ycbcr(img_restored, y_only=True)

        # calculate PSNR and SSIM
        psnr = calculate_psnr(img_gt * 255, img_restored * 255, crop_border=crop_border, input_order='HWC')
        ssim = calculate_ssim(img_gt * 255, img_restored * 255, crop_border=crop_border, input_order='HWC')
        print(f'{i+1:3d}: {basename:25}. \tPSNR: {psnr:.6f} dB, \tSSIM: {ssim:.6f}')
        psnr_all.append(psnr)
        ssim_all.append(ssim)

    print(gt_path)
    print(restored_path)
    print(f'Average: PSNR: {sum(psnr_all) / len(psnr_all):.6f} dB, SSIM: {sum(ssim_all) / len(ssim_all):.6f}')


# Example usage:
gt_path = r'D:\demo\gt'
restored_path = r'D:\demo\lr'
calculate_psnr_ssim(gt_path, restored_path, crop_border=0, suffix='', test_y_channel=False, correct_mean_var=False)


Testing RGB channels.
  1: 3                        . 	PSNR: 24.742910 dB, 	SSIM: 0.740794
D:\demo\gt
D:\demo\lr
Average: PSNR: 24.742910 dB, SSIM: 0.740794


In [21]:
import cv2
import numpy as np
from os import path as osp
from torchvision.transforms.functional import normalize
from basicsr.metrics import calculate_psnr, calculate_ssim
from basicsr.utils import bgr2ycbcr, scandir, img2tensor

try:
    import lpips
except ImportError:
    print('Please install lpips: pip install lpips')


def calculate_lpips(img_gt, img_restored, loss_fn_vgg, mean, std):
    img_gt, img_restored = img2tensor([img_gt, img_restored], bgr2rgb=True, float32=True)
    # norm to [-1, 1]
    normalize(img_gt, mean, std, inplace=True)
    normalize(img_restored, mean, std, inplace=True)

    # calculate lpips
    lpips_val = loss_fn_vgg(img_restored.unsqueeze(0).cuda(), img_gt.unsqueeze(0).cuda())

    return lpips_val


def calculate_psnr_ssim(gt_path, restored_path, crop_border=0, suffix='', test_y_channel=False, correct_mean_var=False):
    """Calculate PSNR, SSIM, and LPIPS for images.

    Args:
        gt_path (str): Path to gt (Ground-Truth).
        restored_path (str): Path to restored images.
        crop_border (int): Crop border for each side.
        suffix (str): Suffix for restored images.
        test_y_channel (bool): If True, test Y channel (In MatLab YCbCr format). If False, test RGB channels.
        correct_mean_var (bool): Correct the mean and var of restored images.
    """
    psnr_all = []
    ssim_all = []
    lpips_all = []
    img_list_gt = sorted(list(scandir(gt_path, recursive=True, full_path=True)))
    img_list_restored = sorted(list(scandir(restored_path, recursive=True, full_path=True)))

    if test_y_channel:
        print('Testing Y channel.')
    else:
        print('Testing RGB channels.')

    # Configurations for LPIPS
    loss_fn_vgg = lpips.LPIPS(net='vgg').cuda()  # RGB, normalized to [-1,1]
    mean = [0.5, 0.5, 0.5]
    std = [0.5, 0.5, 0.5]

    for i, img_path in enumerate(img_list_gt):
        basename, ext = osp.splitext(osp.basename(img_path))
        img_gt = cv2.imread(img_path, cv2.IMREAD_UNCHANGED).astype(np.float32) / 255.
        # 单通道图片，转为三通道
        if img_gt.ndim == 2:
            img_gt = np.repeat(np.expand_dims(img_gt, axis=2), 3, axis=2)
        if suffix == '':
            img_path_restored = img_list_restored[i]
        else:
            img_path_restored = osp.join(restored_path, basename + suffix + ext)
        img_restored = cv2.imread(img_path_restored, cv2.IMREAD_UNCHANGED).astype(np.float32) / 255.
        # 单通道图片，转为三通道
        if img_restored.ndim == 2:
            img_restored = np.repeat(np.expand_dims(img_restored, axis=2), 3, axis=2)

        if correct_mean_var:
            mean_l = []
            std_l = []
            for j in range(3):
                mean_l.append(np.mean(img_gt[:, :, j]))
                std_l.append(np.std(img_gt[:, :, j]))
            for j in range(3):
                # correct twice
                mean = np.mean(img_restored[:, :, j])
                img_restored[:, :, j] = img_restored[:, :, j] - mean + mean_l[j]
                std = np.std(img_restored[:, :, j])
                img_restored[:, :, j] = img_restored[:, :, j] / std * std_l[j]

                mean = np.mean(img_restored[:, :, j])
                img_restored[:, :, j] = img_restored[:, :, j] - mean + mean_l[j]
                std = np.std(img_restored[:, :, j])
                img_restored[:, :, j] = img_restored[:, :, j] / std * std_l[j]

        if test_y_channel and img_gt.ndim == 3 and img_gt.shape[2] == 3:
            img_gt = bgr2ycbcr(img_gt, y_only=True)
            img_restored = bgr2ycbcr(img_restored, y_only=True)

        # calculate PSNR and SSIM
        psnr = calculate_psnr(img_gt * 255, img_restored * 255, crop_border=crop_border, input_order='HWC')
        ssim = calculate_ssim(img_gt * 255, img_restored * 255, crop_border=crop_border, input_order='HWC')
        lpips_val = calculate_lpips(img_gt, img_restored, loss_fn_vgg, mean, std)

        print(f'{i+1:3d}: {basename:25}. \tPSNR: {psnr.item():.6f} dB, \tSSIM: {ssim.item():.6f}, \tLPIPS: {lpips_val.item():.6f}')

        psnr_all.append(psnr)
        ssim_all.append(ssim)
        lpips_all.append(lpips_val)

    print(gt_path)
    print(restored_path)
    average_psnr = sum(psnr_all) / len(psnr_all)
    average_ssim = sum(ssim_all) / len(ssim_all)
    average_lpips = sum(lpips_all).item() / len(lpips_all)  # 使用 .item() 转换为 Python 标量

    print(f'Average: PSNR: {average_psnr:.6f} dB, SSIM: {average_ssim:.6f}, LPIPS: {average_lpips:.6f}')



# Example usage:
gt_path = r'O:\MICCAI2024\Real-Motion-01\local_test\datasets\run02\gt'
restored_path = r'O:\MICCAI2024\Real-Motion-01\local_test\datasets\run02\acc4'
calculate_psnr_ssim(gt_path, restored_path, crop_border=0, suffix='', test_y_channel=False, correct_mean_var=False)


Testing RGB channels.
Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
Loading model from: e:\anaconda3\envs\basicSR\lib\site-packages\lpips\weights\v0.1\vgg.pth
  1: 0                        . 	PSNR: 25.108554 dB, 	SSIM: 0.657859, 	LPIPS: 0.353985
  2: 1                        . 	PSNR: 24.988664 dB, 	SSIM: 0.651560, 	LPIPS: 0.368426
  3: 10                       . 	PSNR: 23.616713 dB, 	SSIM: 0.581023, 	LPIPS: 0.369207
  4: 11                       . 	PSNR: 23.343667 dB, 	SSIM: 0.576714, 	LPIPS: 0.366536
  5: 12                       . 	PSNR: 23.148944 dB, 	SSIM: 0.566546, 	LPIPS: 0.369623
  6: 13                       . 	PSNR: 22.776741 dB, 	SSIM: 0.566349, 	LPIPS: 0.371713
  7: 14                       . 	PSNR: 23.448898 dB, 	SSIM: 0.626461, 	LPIPS: 0.353712
  8: 15                       . 	PSNR: 23.799406 dB, 	SSIM: 0.609945, 	LPIPS: 0.363878
  9: 16                       . 	PSNR: 24.201500 dB, 	SSIM: 0.641254, 	LPIPS: 0.352771
 10: 17                       . 	